**Source table, which will be cloned later on**

In [0]:
%python 
dbutils.fs.rm("dbfs:/user/hive/warehouse/source",True)

True

In [0]:
%sql
DROP TABLE IF EXISTS source;
CREATE OR REPLACE TABLE  source (id INT, content STRING);

INSERT INTO source VALUES (1,"a"),(2,"b"),(3,"c"),(4,"d");
INSERT INTO source VALUES (5,"e");

DELETE FROM source
WHERE id=5;

ALTER TABLE source  ADD CONSTRAINT Ids CHECK (id>0)

In [0]:
%sql
DESC HISTORY source

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
4,2025-02-03T08:00:49Z,5755764547042441,inaya998877@gmail.com,ADD CONSTRAINT,"Map(name -> Ids, expr -> id > 0)",null,List(4318753886016008),0203-052121-nggyovx3,3,WriteSerializable,false,Map(),null,Databricks-Runtime/15.4.x-scala2.12
3,2025-02-03T08:00:47Z,5755764547042441,inaya998877@gmail.com,DELETE,"Map(predicate -> [""(id#198327 = 5)""])",null,List(4318753886016008),0203-052121-nggyovx3,2,WriteSerializable,false,"Map(numRemovedFiles -> 1, numRemovedBytes -> 691, numCopiedRows -> 0, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 1292, numDeletionVectorsUpdated -> 0, numDeletedRows -> 1, scanTimeMs -> 814, numAddedFiles -> 0, numAddedBytes -> 0, rewriteTimeMs -> 478)",null,Databricks-Runtime/15.4.x-scala2.12
2,2025-02-03T08:00:43Z,5755764547042441,inaya998877@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(4318753886016008),0203-052121-nggyovx3,1,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 691)",null,Databricks-Runtime/15.4.x-scala2.12
1,2025-02-03T08:00:42Z,5755764547042441,inaya998877@gmail.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(4318753886016008),0203-052121-nggyovx3,0,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 4, numOutputBytes -> 712)",null,Databricks-Runtime/15.4.x-scala2.12
0,2025-02-03T08:00:40Z,5755764547042441,inaya998877@gmail.com,CREATE OR REPLACE TABLE,"Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> true, properties -> {}, statsOnLoad -> false)",null,List(4318753886016008),0203-052121-nggyovx3,null,WriteSerializable,true,Map(),null,Databricks-Runtime/15.4.x-scala2.12


In [0]:
%sql
DROP TABLE IF EXISTS deep_clone;

CREATE OR REPLACE TABLE deep_clone
  DEEP CLONE source;

source_table_size,source_num_of_files,num_of_synced_transactions,num_removed_files,num_copied_files,removed_files_size,copied_files_size
712,1,null,1,1,818,712


In [0]:
%sql
SELECT * FROM deep_clone

id,content
1,a
2,b
3,c
4,d


In [0]:
%sql
DESC HISTORY deep_clone

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
1,2025-02-03T08:00:59Z,5755764547042441,inaya998877@gmail.com,CLONE,"Map(source -> spark_catalog.default.source, sourceVersion -> 4, isShallow -> false)",null,List(4318753886016008),0203-052121-nggyovx3,0,Serializable,false,"Map(removedFilesSize -> 818, numRemovedFiles -> 1, sourceTableSize -> 712, numCopiedFiles -> 1, copiedFilesSize -> 712, sourceNumOfFiles -> 1)",null,Databricks-Runtime/15.4.x-scala2.12
0,2025-01-29T13:05:06Z,5755764547042441,inaya998877@gmail.com,CLONE,"Map(source -> spark_catalog.default.source, sourceVersion -> 4, isShallow -> false)",null,List(4318753886016008),0129-070100-p6rjfh7p,-1,Serializable,false,"Map(removedFilesSize -> 0, numRemovedFiles -> 0, sourceTableSize -> 818, numCopiedFiles -> 1, copiedFilesSize -> 818, sourceNumOfFiles -> 1)",null,Databricks-Runtime/12.2.x-scala2.12


In [0]:
%fs ls dbfs:/user/hive/warehouse/deep_clone

path,name,size,modificationTime
dbfs:/user/hive/warehouse/deep_clone/_delta_log/,_delta_log/,0,0
dbfs:/user/hive/warehouse/deep_clone/part-00000-43ab773f-fe79-4cd7-accf-4dceb2d01f25-c000.snappy.parquet,part-00000-43ab773f-fe79-4cd7-accf-4dceb2d01f25-c000.snappy.parquet,712,1738569655000
dbfs:/user/hive/warehouse/deep_clone/part-00000-5dec0b63-3956-483f-a54e-3fa4d6ea6d5f-c000.snappy.parquet,part-00000-5dec0b63-3956-483f-a54e-3fa4d6ea6d5f-c000.snappy.parquet,818,1738155904000


In [0]:
%sql
INSERT INTO source VALUES (5,"e");
SELECT * FROM deep_clone;

id,content
1,a
2,b
3,c
4,d


In [0]:
%sql
DELETE FROM source;
SET spark.databricks.delta.retentionDurationCheck.enabled=false;
VACUUM source RETAIN 0 HOURS;

SELECT * FROM deep_clone;

id,content
1,a
2,b
3,c
4,d


In [0]:
%sql
SET spark.databricks.delta.retentionDurationCheck.enabled=True;

key,value
spark.databricks.delta.retentionDurationCheck.enabled,True


**The benefit of deep clone is , any changes made to source table wont affect the deep_clone as it copies full metadata,values and everything**

#Now start fresh using shallow_clone

In [0]:
%sql
DROP TABLE IF EXISTS source;
CREATE OR REPLACE TABLE  source (id INT, content STRING);

INSERT INTO source VALUES (1,"a"),(2,"b"),(3,"c"),(4,"d");
INSERT INTO source VALUES (5,"e");
UPDATE source SET content="f" WHERE id=4;
 
ALTER TABLE source  ADD CONSTRAINT Ids CHECK (id>0)

**Shallow clone**

In [0]:
%sql
DROP TABLE IF EXISTS shallow_clone;
CREATE OR REPLACE TABLE shallow_clone
  SHALLOW CLONE source;

source_table_size,source_num_of_files,num_of_synced_transactions,num_removed_files,num_copied_files,removed_files_size,copied_files_size
1403,2,null,2,0,1615,0


In [0]:
%sql
DESC HISTORY shallow_clone;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
1,2025-02-03T08:01:41Z,5755764547042441,inaya998877@gmail.com,CLONE,"Map(source -> spark_catalog.default.source, sourceVersion -> 4, isShallow -> true)",null,List(4318753886016008),0203-052121-nggyovx3,0,Serializable,false,"Map(removedFilesSize -> 1615, numRemovedFiles -> 2, sourceTableSize -> 1403, numCopiedFiles -> 0, copiedFilesSize -> 0, sourceNumOfFiles -> 2)",null,Databricks-Runtime/15.4.x-scala2.12
0,2025-01-30T05:32:08Z,5755764547042441,inaya998877@gmail.com,CLONE,"Map(source -> spark_catalog.default.source, sourceVersion -> 4, isShallow -> true)",null,List(4318753886016008),0130-050659-ysdddoy1,-1,Serializable,false,"Map(removedFilesSize -> 0, numRemovedFiles -> 0, sourceTableSize -> 1615, numCopiedFiles -> 0, copiedFilesSize -> 0, sourceNumOfFiles -> 2)",null,Databricks-Runtime/12.2.x-scala2.12


In [0]:
%sql
DESC DETAIL shallow_clone

format,id,name,description,location,createdAt,lastModified,partitionColumns,clusteringColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics
delta,3fdd178c-93b2-4857-b071-b66506e35581,spark_catalog.default.shallow_clone,null,dbfs:/user/hive/warehouse/shallow_clone,2025-02-03T08:01:27.771Z,2025-02-03T08:01:41Z,List(),List(),2,1403,Map(delta.constraints.ids -> id > 0),1,3,"List(appendOnly, checkConstraints, invariants)",Map()


In [0]:
%fs ls dbfs:/user/hive/warehouse/shallow_clone

path,name,size,modificationTime
dbfs:/user/hive/warehouse/shallow_clone/_delta_log/,_delta_log/,0,0


In [0]:
%sql
INSERT INTO source VALUES (5,"NEW member in the data");
SELECT * FROM  shallow_clone;

id,content
1,a
2,b
3,c
4,f
5,e


In [0]:
%sql
DELETE FROM source;
SET spark.databricks.delta.retentionDurationCheck.enabled=false;
VACUUM source RETAIN 0 HOURS;

path
dbfs:/user/hive/warehouse/source


In [0]:
%sql
SELECT * FROM shallow_clone;

org.apache.spark.SparkException: [FAILED_READ_FILE.DBR_FILE_NOT_EXIST] Error while reading file dbfs:/user/hive/warehouse/source/part-00000-5665702c-1a07-4114-8f88-c448e92252dc-c000.snappy.parquet. [DELTA_SHALLOW_CLONE_FILE_NOT_FOUND] File dbfs:/user/hive/warehouse/source/part-00000-5665702c-1a07-4114-8f88-c448e92252dc-c000.snappy.parquet referenced in the transaction log cannot be found. This can occur when data has been manually deleted from the file system rather than using the table `DELETE` statement. This table appears to be a shallow clone, if that is the case, this error can occur when the original table from which this table was cloned has deleted a file that the clone is still using. If you want any clones to be independent of the original table, use a DEEP clone instead. SQLSTATE: KD001
	at org.apache.spark.sql.errors.QueryExecutionErrors$.fileNotExistErrorDBR(QueryExecutionErrors.scala:1069)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.logMissi

### - Shallow clone is faster
### - It is cheaper it doesnt consume so much storage as deep clone
### - Note:- It Relies on the data of the source table
